* https://www.youtube.com/watch?v=1rzUbimPzpE

* Requires API key
- 100 free/day
- over 100/day: $5/1000 queries or $0.001/search

* https://aistudio.google.com/

In [ ]:
!pip install -U google-generativeai>=0.8.3

# Gemini Grounding with Google Search

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_AI_STUDIO') # 'GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for model in genai.list_models():
  if '002' in model.name:
    print(model.name)

models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


## The Search Tool

`GoogleSearchRetrieval` is a tool to do a search and retrieve public webpages for grounding. Part of `GoogleSearchRetrieval` is dynamic search retrieval, which uses real time retrieved data to build answers to queries.





In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')

response = model.generate_content(contents="Who won the Nobal prive in physics in 2024?",
                                  tools='google_search_retrieval')

## Dealing with 429 errors

If you're getting errors on this part and they're 429 errors, usually that relates to the fact that you don't have any quota set up. And that basically just means that you haven't setup a billing account here.

 Resource has been exhausted (e.g. check quota).

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

The 2024 Nobel Prize in Physics was awarded to John J. Hopfield and Geoffrey E. Hinton "for foundational discoveries and inventions that enable machine learning with artificial neural networks".  Hinton is known as "a godfather of AI" for his work on methods enabling autonomous data analysis.


In [ ]:
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

In [ ]:
links = response.candidates[0].grounding_metadata.grounding_chunks
links

[web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfwGBTuFt1A_KKeuRE7AiswQgaPWI9usfZES-jI10K-c-0aMVWhhoX71Sbv37mcY6pApwJQrVEC3h8EaAmD1lO6j_xidcXXSe-OR1f4NDvqYshidbsVMwyDmJLfHJqNRnp8JelOZVnNKQ9jMoxcwA8htdOPu-b8Uyg=="
  title: "aip.org"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfwiQneKgSdmvyViF4dyqrIsh1R3m2YvJTFexms4hfGNlpfbROOijRYnxBOjM4g0ET_q40Gm0Jq4wLEqX_U1vhYvHMxSt3i1iaNoIOwP-otnthDf6P909mjr-TynVybjJRt0tps6Hfs="
  title: "youtube.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AZnLMfwKUM-WucPPJSEzzArXOVaZg_YyTNX3TXWj3XXelBp9DdKJl517aLIZWIdLd2JlgzAySlWGXu3gAxxm3XHOH2iWEzniXmYHYNETyD67KsZMMlCZAx99KaudIsPpjHBoxyNJ51_0HVcqr9pApMZF2ye5V9x0gvBTGyCCzZM4"
  title: "weforum.org"
}
]

In [ ]:

links = [(chunk.web.title, chunk.web.uri) for chunk in links]

html_links = '<br>'.join(f'<a href="{url}" target="_blank">{title}</a>' for title, url in links)
display(HTML(html_links))

In [ ]:
response.candidates[0].grounding_metadata.grounding_supports

[segment {
  end_index: 194
  text: "The 2024 Nobel Prize in Physics was awarded to John J. Hopfield and Geoffrey E. Hinton \"for foundational discoveries and inventions that enable machine learning with artificial neural networks\"."
}
grounding_chunk_indices: 0
grounding_chunk_indices: 1
confidence_scores: 0.9747981429100037
confidence_scores: 0.9693813920021057
, segment {
  start_index: 196
  end_index: 293
  text: "Hinton is known as \"a godfather of AI\" for his work on methods enabling autonomous data analysis."
}
grounding_chunk_indices: 2
confidence_scores: 0.9746900796890259
]

### Customize search grounding parameters

For a dictionary implementation, you don't actually have to pass in `mode` or `dynamic_threshold` key-value pairs. If you don't pass in a value, a default value is created. See the next code cell for this. For the `mode` value, you can pass in one of two specifications:

1. `unspecified`: Always trigger retrieval.
2. `dynamic`: Run retrieval only when system decides it is necessary.

The `dynamic_threshold` is a float value that acts as a threshold on how relevant answers returned should be. You can tweak this parameter depending on how much variation you want in your answer.

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="Who is Geoffrey Hinton?",
                                  tools={"google_search_retrieval": {"dynamic_retrieval_config": {"mode": "MODE_DYNAMIC",
                                                                                                  "dynamic_threshold": 0.5}}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Geoffrey Hinton is a British-Canadian cognitive psychologist and computer scientist renowned for his groundbreaking work in artificial intelligence, particularly in the area of artificial neural networks.  He is often called the "Godfather of AI" for his significant contributions to the field.  Born on December 6, 1947, Hinton's career has been marked by his persistent exploration of neural networks, even when the field was not popular.  His research has revolutionized machine learning and paved the way for many modern AI applications.

Hinton earned a degree in experimental psychology from the University of Cambridge in 1970 and a PhD in Artificial Intelligence from the University of Edinburgh in 1978.  He held positions at various universities, including the University of Sussex, University of California, San Diego, Carnegie Mellon University, and University College London.  He joined the University of Toronto in 1987 and is currently a University Professor Emeritus there.

Some of Hinton's key contributions include:

* **Backpropagation:**  Along with David Rumelhart and Ronald J. Williams, Hinton popularized the backpropagation algorithm, a key process for training multi-layer neural networks. This 1986 paper was highly influential in the development of the field.
* **Boltzmann Machines:** In 1983, he co-invented Boltzmann machines, one of the first neural networks capable of learning internal representations.
* **Deep Learning:**  Hinton's research group at the University of Toronto made major breakthroughs in deep learning that transformed speech recognition and object classification.
* **AlexNet:** In 2012, with his students Alex Krizhevsky and Ilya Sutskever, Hinton developed AlexNet, an eight-layer neural network that significantly outperformed other image recognition programs. This marked a major breakthrough in computer vision.
* **Word Embeddings:** Hinton was the first to use backpropagation for learning word embeddings, further advancing natural language processing.

Hinton has received numerous prestigious awards, including the 2018 Turing Award (often referred to as the "Nobel Prize of Computing"), the 2010 Gerhard Herzberg Canada Gold Medal, and most recently, the 2024 Nobel Prize in Physics for his foundational discoveries and inventions that enable machine learning with artificial neural networks.  He co-founded DNNresearch, which was acquired by Google, where he worked until 2023. He also co-founded and is the Chief Scientific Advisor of the Vector Institute in Toronto. In 2023, Hinton publicly expressed concerns about the risks of AI technology, becoming a prominent voice warning about the potential dangers of advanced AI systems.


In [ ]:
HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

In [ ]:
links = response.candidates[0].grounding_metadata.grounding_chunks

links = [(chunk.web.title, chunk.web.uri) for chunk in links]

html_links = '<br>'.join(f'<a href="{url}" target="_blank">{title}</a>' for title, url in links)
display(HTML(html_links))

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-pro-002')
response = model.generate_content(contents="Who is Geoffrey Hinton?",
                                  tools={"google_search_retrieval": {}})

In [ ]:
Markdown(response.candidates[0].content.parts[0].text)

Geoffrey Hinton is a British-Canadian cognitive psychologist and computer scientist known as the "Godfather of AI" for his revolutionary work on artificial neural networks.  Born in Wimbledon, London, on December 6, 1947, he currently holds the title of University Professor Emeritus at the University of Toronto.

Hinton's academic journey began at the University of Cambridge, where he explored physiology, philosophy, and physics before graduating with a degree in experimental psychology in 1970. He then pursued a Ph.D. in Artificial Intelligence (AI) from the University of Edinburgh, graduating in 1978.  Despite discouragement from his professors, Hinton persisted in his research on neural networks, inspired by the structure of the human brain.

Hinton's career has included positions at the University of Sussex, the MRC Applied Psychology Unit, the University of California, San Diego, and Carnegie Mellon University. In 1987, he joined the University of Toronto, where he remains affiliated today.  He was also the founding director of the Gatsby Charitable Foundation Computational Neuroscience Unit at University College London.  From 2013 to 2023, he divided his time between the University of Toronto and Google Brain, leaving Google to speak openly about the risks of AI.

Hinton's key contributions to AI include:

* **Backpropagation:** Co-authoring a landmark 1986 paper popularizing the backpropagation algorithm for training multi-layer neural networks.
* **Boltzmann Machines:** Co-inventing Boltzmann machines in 1985 with David Ackley and Terry Sejnowski.
* **Deep Learning:** Making significant advancements in deep learning, leading to breakthroughs in speech recognition and object classification.
* **AlexNet:** Developing AlexNet in 2012 with his students, a deep learning model that significantly advanced image recognition.
* **Word Embeddings:** Being the first to use backpropagation for learning word embeddings.

Hinton's work has been widely recognized with prestigious awards, including the 2018 Turing Award, often called the "Nobel Prize of Computing," and the 2024 Nobel Prize in Physics, shared with John Hopfield. He is the first person to win both the Turing Award and the Nobel Prize in Physics.  He also holds the Gerhard Herzberg Canada Gold Medal (2010) and the Royal Society's Royal Medal (2022).  He is a fellow of the Royal Society (UK), the Royal Society of Canada, and an international member of the US National Academy of Sciences, the US National Academy of Engineering, and the American Academy of Arts and Sciences. Since 2017, he has served as the Chief Scientific Advisor at the Vector Institute in Toronto.


## Doing it all in Chat mode

In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-flash-002',
                              tools="google_search_retrieval")

chat = model.start_chat()
result = chat.send_message('Who is Geoffrey Hinton?')

In [ ]:
display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Geoffrey Everest Hinton (born December 6, 1947) is a highly influential British-Canadian computer scientist and cognitive psychologist.  He is widely recognized as a pioneer in the field of artificial intelligence (AI), specifically deep learning, earning him the moniker "Godfather of AI".

Hinton's contributions to AI are substantial and groundbreaking.  He co-authored a highly influential 1986 paper that popularized the backpropagation algorithm for training multi-layer neural networks, a crucial advancement in the field.  He also played a pivotal role in the development of Boltzmann machines, a type of neural network capable of learning internal representations.  His research group at the University of Toronto made significant breakthroughs in deep learning, revolutionizing areas such as speech recognition and object classification, notably through the development of AlexNet in 2012 with his students. AlexNet dramatically improved image recognition accuracy.

His academic career has spanned several prestigious institutions, including the University of Toronto (where he is currently University Professor Emeritus),  University College London, Carnegie Mellon University, and Google (Google Brain), where he worked until May 2023.  He is also the Chief Scientific Advisor at the Vector Institute in Toronto.  Hinton's extensive accolades include the 2018 Turing Award (often called the "Nobel Prize of Computing") and the 2024 Nobel Prize in Physics, awarded jointly with John J. Hopfield, for their foundational discoveries and inventions enabling machine learning with artificial neural networks.  He has also received numerous other awards and recognitions throughout his career.

In recent years, Hinton has become increasingly vocal about the potential risks and dangers associated with rapidly advancing AI technology.  His concerns highlight the ethical and societal implications of AI, emphasizing the need for cautious development and responsible implementation.


In [ ]:
result = chat.send_message('what is the google stock price?')


display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

The provided sources give slightly different prices for Alphabet Inc. (Google) Class C stock (GOOG), reflecting the fluctuation of stock prices throughout the day.  There is also a distinction made between Class A (GOOGL) and Class C (GOOG) shares.  The question doesn't specify which class of shares is being inquired about, but I will provide information for both based on the data available.

As of the closing prices on Friday, November 8th, 2024:

* **GOOG (Class C):**  The closing price was approximately $179.93 according to one source. Other sources show prices slightly different, between $179.57 and $182.35 as the range of the day's trading.  This reflects the constant change in stock prices.

* **GOOGL (Class A):**  The provided text doesn't give the exact current price for GOOGL (Class A shares).  To get the most up-to-date price, you should consult a live stock ticker.


Please note that stock prices are constantly changing, and these values are only accurate as of the dates and times reflected in the provided sources.  For the most current information, you should refer to a live stock quote source.


In [ ]:
model = genai.GenerativeModel('models/gemini-1.5-flash-002')
chat = model.start_chat()
result = chat.send_message('who won the Nobel prize for physics this year?',
                           tools="google_search_retrieval")

In [ ]:
display(Markdown(result.text))

display(HTML(result.candidates[0].grounding_metadata.search_entry_point.rendered_content))

The 2024 Nobel Prize in Physics was awarded to John J. Hopfield and Geoffrey E. Hinton.  They were recognized for their foundational discoveries and inventions that enabled machine learning using artificial neural networks.  Hopfield's contribution involved creating an associative memory capable of storing and reconstructing patterns in data, while Hinton invented a method for autonomously identifying properties within data, crucial for tasks like identifying elements in images.
